Created by Angelo Orletti Del Rey for its masters in psychiatry at Unifesp - SP - Brasil

It was based in our groups previeous works in cotical striatal connectivity in INPD data base of fmri

In [1]:
import numpy as np
import os, sys
import bids
import nibabel as nib
from scipy.stats import pearsonr
from scipy.stats import chi2
import statsmodels.api as sm
import statsmodels.formula.api as smf
from nilearn import plotting as nplot
from nilearn import image as nimg
from nilearn.image import resample_to_img
import matplotlib.pyplot as plt
import argparse
import pandas as pd

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from scipy.linalg import LinAlgError

# Suppress convergence warnings
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="statsmodels")


c:\Users\angel\Documents\INPD_neuroimages_CAPE\.venv_fmriINPD\lib\site-packages\nilearn\__init__.py:67: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()


In [3]:
def parse():

        options = argparse.ArgumentParser(description="Run 1st level analysis. Created by ...")
        options.add_argument('-p', '--participants', nargs='+',dest="participants", action='store', type=str, required=False,
                            help='id of subject or list of subjects')
        options.set_defaults(participants=None)
        options.add_argument('-w', '--workdir',dest="workdir", action='store', type=str, required=False,
                            help='the work directory for the project')
        options.set_defaults(workdir=os.environ["ROOTDIR"])
        options.add_argument('-b', '--bidsdir',dest="rawdata", action='store', type=str, required=False,
                            help='the work directory for the project')
        options.set_defaults(rawdata=os.path.join(os.environ["ROOTDIR"],"BIDS"))
        options.add_argument('-d', '--derivatives',dest="derivatives", action='store', type=str, required=True,
                            help='path to fMRIprep directory')
        options.add_argument('-o', '--outputs',dest="output", action='store', type=str, required=True,
                            help='path to fMRIprep directory')
        #print(options.parse_args())

        return options.parse_args()

## CHANGE HERE THE PATH TO YOUR DATA
os.environ["ROOTDIR"] = r'C://Users//angel\Documents//INPD_neuroimages_CAPE'   # seth path
rootdir = os.environ["ROOTDIR"]
if hasattr(sys, "ps1"):
    options = {}
    workdir = os.environ["ROOTDIR"]
    firstleveldir  = os.path.join(workdir,"BIDS","derivatives","first_level_results")
    demographic = os.path.join(workdir,"metadata")
    confounds = os.path.join(workdir,"metadata")
    output = os.path.join(workdir,'sensitivity_analisys_second_level')
    participants = []

else :
    options = parse()
    participants = options.participants
    workdir = options.workdir
    rawdata = options.rawdata
    derivat = options.derivatives
    output  = options.outputs

print('firstlevel: ', firstleveldir)
    
bidslayout = bids.BIDSLayout(firstleveldir, validate = False) #With validate = True it doesn't find any subjects

if not participants:
    participants = bidslayout.get_subjects() #take all subjects
    #participants_sub = ["sub-" + item for item in participants]

seednames = ['DCPutamen',
            'DorsalCaudate',
            'DRPutamen',
            'InfVentralCaudate',
            'SupVentralCaudate',
            'VRPutamen'
            ]

tmap_allsubj = [] #initialize array for tmaps

print("Loading tmaps for all participants...")
for p in participants:
    #print(f"Subject: {p}")
    p = p.replace("sub-", "")

    # for ses in bidslayout.get_sessions(subject=p):
    #     #print(f"Session: {ses}")                
    
    ses = '1'

    for r in bidslayout.get_runs(subject=p, session=ses):
        #print(f"Run: {r}")

        tmap_metadata = {
            'task': 'rest',
            'suffix': 'tstat',
            'extension': '.txt',
            'run': str(r),
            'session': str(ses),
            'subject': 'sub-'+p,
            'space': 'MNI152NLin2009cAsym',
            'seed': 'SeedtoROI'
        }

        # Save each tmap as a nifti file
        filepath = os.path.join(firstleveldir,tmap_metadata["subject"],"ses-"+tmap_metadata['session'],"func")
        filename = tmap_metadata["subject"] + "_" + \
                    "ses-"+tmap_metadata['session'] + "_" + \
                    "task-"+tmap_metadata['task'] + "_" + \
                    "run-"+tmap_metadata['run'] + "_" + \
                    "space-"+tmap_metadata['space'] + "_" + \
                    "seed-"+tmap_metadata['seed'] + "_" + \
                    tmap_metadata["suffix"] + \
                    tmap_metadata["extension"]
        tmap_path = os.path.join(filepath, filename)

        #load the tmaps for each subject and then append to the tmap_allsubj array
        tmap = pd.read_csv(tmap_path, delimiter=' ', header=None, skiprows=1) #also skips the first row
        tmap.columns = seednames
        tmap_allsubj.append(tmap)


firstlevel:  C://Users//angel\Documents//INPD_neuroimages_CAPE\BIDS\derivatives\first_level_results
Loading tmaps for all participants...


In [ ]:
#load socioeconomic and psichometric data for all subjects
print("Loading socioeconomic and psichometric data for all participants...")
socio_psi=pd.read_csv(os.path.join(demographic,'variaveis_analise_conf_nova-exclusão.tsv'), sep='\t')
#substitute in age colunm , to . for decimals
socio_psi['age'] = socio_psi['age'].str.replace(',', '.').astype(float)

print('loading the p factor scores')
p_factor = pd.read_excel(os.path.join(demographic,'factor-loading-angelo.xlsx'))

print('Merging the two dataframes')
# correct the collumn ident of p_factor to have the format 'sub-XXXXX'
p_factor['ident'] = p_factor['ident'].apply(lambda x: f'sub-{int(x):05d}')
# rename the collumn ident to sub_id
p_factor = p_factor.rename(columns={'ident': 'sub_id'})
# merge the two dataframes on sub_id
socio_psi = pd.merge(socio_psi, p_factor[['sub_id', 'cl_p_fl']], on='sub_id', how='left')

Loading socioeconomic and psichometric data for all participants...
loading the p factor scores
Merging the two dataframes


In [8]:
# organinzing the tmaps. Joining the columns of the tmaps for all subjects. So for each subject
# we have to join the first collunm of the first tmap with the first column of the second tmap and so on
# and then join the second column of the first tmap with the second column of the second tmap and so on
# and so on
print("Organizing tmaps...")
tmap_allsubj = pd.concat(tmap_allsubj, axis=1)
tmap_allsubj_organized = {}

for seed in seednames:
    #drop columns with names different from seednames[i]
    tmap_allsubj_organized[seed] = tmap_allsubj.drop(columns=[col for col in tmap_allsubj.columns if col != seed])

# What do we have now: tmap_allsubj_organized is a dictionary with keys being the seednames and values being
# dataframes with the tmaps for all subjects for that seed. This have dimensions of n_ROIs x n_subjects
# Now we have to transpose it to get the data in the format n_subjects x n_ROIs and then join with the
# socioeconomic and psichometric data
# Also, we have to add the sub_id column to the conectivity data to be able to join with the socioeconomic
# and psichometric data
subs = {'sub_id': []}
for p in participants:
    ses = '1'
    if bidslayout.get_runs(subject=p, session=ses) != []:
        subs['sub_id'].append('sub-'+p)

subs = pd.DataFrame(subs)

# create the collunms names for the data frame of rois
roi_names = [f'roi_{i}' for i in range(101)]

for seed in seednames:
    tmap_allsubj_organized[seed] = tmap_allsubj_organized[seed].transpose()
    tmap_allsubj_organized[seed].columns = roi_names
    tmap_allsubj_organized[seed] = tmap_allsubj_organized[seed].reset_index().join(subs, how='inner')
    tmap_allsubj_organized[seed] = tmap_allsubj_organized[seed].drop(columns=['index'])

# Now we have to join the tmaps with the socioeconomic and psichometric data
print("Joining tmaps with socioeconomic and psichometric data...")

for seed in seednames:
    tmap_allsubj_organized[seed] = pd.merge(tmap_allsubj_organized[seed], socio_psi, on='sub_id', how='inner')
    tmap_allsubj_organized[seed] = tmap_allsubj_organized[seed].drop(columns=['Unnamed: 0'])

# now we run the regression analysis for each seed. The dependent variable is the connectivity and the independent
# variables are the CAPE scores. The socioeconomic variables are used as confund variables
print("Running regression analysis (salience and defualt mode network)...")

shaeffer_network = pd.read_csv('tpl-MNI152NLin2009cAsym_atlas-Schaefer2018_desc-100Parcels7Networks_dseg.tsv', delimiter='\t')

#extract the index with the name with SalVentAttn - salience network
salience_index = shaeffer_network[shaeffer_network['name'].str.contains('SalVentAttn')]
#extract the index with the name with Default - default mode network
default_index = shaeffer_network[shaeffer_network['name'].str.contains('Default')]

# running the regression analysis for each roi in the salience network
results_salience, results_default = {}, {}
for seed in seednames:
    print(f'running for cl_p_fl e seed {seed}')
    results_salience[seed], results_default[seed] = {}, {}

    #salience analisys
    for roi in salience_index['index']:

        #building my strign for the fomrula
        formula_salience = f'cl_p_fl ~ roi_{roi} + age + gender + abepscore + colection_site + fd_count_high_value'

        #running the regression analysis
        model_salience = smf.ols(formula_salience, data=tmap_allsubj_organized[seed])
        fit_salience = model_salience.fit()

        # Extract desired values
        results_salience[seed][roi] = {
            'coef': fit_salience.params[f'roi_{roi}'],
            'intercept': fit_salience.params['Intercept'],
            't_value': fit_salience.tvalues[f'roi_{roi}'],
            'p_value': fit_salience.pvalues[f'roi_{roi}'],
            'R_squared': fit_salience.rsquared,
            'Adj_R_squared': fit_salience.rsquared_adj,
            'AIC': fit_salience.aic,
            'BIC': fit_salience.bic
        }
        
    #DMN analisys
    for roi in default_index['index']:

        #building my strign for the fomrula
        formula_default = f'cl_p_fl ~ roi_{roi} + age + gender + abepscore + colection_site + fd_count_high_value'
        
        # Run the regression analysis
        model_default = smf.ols(formula_default, data=tmap_allsubj_organized[seed])
        fit_default = model_default.fit()

        # Extract desired values
        results_default[seed][roi] = {
            'coef': fit_default.params[f'roi_{roi}'],
            'intercept': fit_default.params['Intercept'],
            't_value': fit_default.tvalues[f'roi_{roi}'],
            'p_value': fit_default.pvalues[f'roi_{roi}'],
            'R_squared': fit_default.rsquared,
            'Adj_R_squared': fit_default.rsquared_adj,
            'AIC': fit_default.aic,
            'BIC': fit_default.bic
        }

Organizing tmaps...
Joining tmaps with socioeconomic and psichometric data...
Running regression analysis (salience and defualt mode network)...
running for cl_p_fl e seed DCPutamen
running for cl_p_fl e seed DorsalCaudate
running for cl_p_fl e seed DRPutamen
running for cl_p_fl e seed InfVentralCaudate
running for cl_p_fl e seed SupVentralCaudate
running for cl_p_fl e seed VRPutamen


In [9]:
#correcting the p-values
from statsmodels.stats.multitest import multipletests
for seed in seednames:
    print(f'Salience - {seed}')
    p_values = []

    for roi in results_salience[seed].keys():
        # print(roi)
        # Extract the p-values for the current seed and roi
        p_values.append(results_salience[seed][roi]['p_value'])

    #correct the p-values using the Benjamini-Hochberg method
    corrected_p_values = multipletests(p_values, method='fdr_bh')[1]

    # Update the results with the corrected p-values
    for index, roi in enumerate(results_salience[seed].keys()):
        results_salience[seed][roi]['corrected_p_value'] = corrected_p_values[index]

    print(f'DMN - {seed}')
    p_values = []

    for roi in results_default[seed].keys():
        # print(roi)
        # Extract the p-values for the current seed and roi
        p_values.append(results_default[seed][roi]['p_value'])

    #correct the p-values using the Benjamini-Hochberg method
    corrected_p_values = multipletests(p_values, method='fdr_bh')[1]

    # Update the results with the corrected p-values
    for index, roi in enumerate(results_default[seed].keys()):
        # print(f'index {index} roi {roi}')
        results_default[seed][roi]['corrected_p_value'] = corrected_p_values[index]

Salience - DCPutamen
DMN - DCPutamen
Salience - DorsalCaudate
DMN - DorsalCaudate
Salience - DRPutamen
DMN - DRPutamen
Salience - InfVentralCaudate
DMN - InfVentralCaudate
Salience - SupVentralCaudate
DMN - SupVentralCaudate
Salience - VRPutamen
DMN - VRPutamen


In [10]:
# exporting the results to a csv file
print("Exporting results to csv files...")

for seed in seednames:
    for roi in results_salience[seed].keys():
        results_salience[seed][roi] = pd.DataFrame(results_salience[seed][roi],index=[0])
        results_salience[seed][roi].to_csv(os.path.join(output, f'cl_p_fl-{seed}-roi_{roi}_2ndlvl.csv'))
    for roi in results_default[seed].keys():
        results_default[seed][roi] = pd.DataFrame(results_default[seed][roi],index=[0])
        results_default[seed][roi].to_csv(os.path.join(output, f'cl_p_fl-{seed}-roi_{roi}_2ndlvl.csv'))
    tmap_allsubj_organized[seed].to_csv(os.path.join(output,f'{seed}_data-2ndlvl.csv'))

Exporting results to csv files...


In [33]:
for seed in seednames:
    print(f"DMN seed-{seed} ")
    for roi in results_default[seed].keys():
        if results_default[seed][roi]["corrected_p_value"].item() <= 0.05:
            print(f"roi-{roi} -> corr_p_val {results_default[seed][roi]['corrected_p_value']}")


for seed in seednames:
    print(f"salience seed-{seed} ")
    for roi, res in results_salience[seed].items():
        val = res.get('corrected_p_value')
        if val is not None and val.item() <= 0.05:
            print(f"roi-{roi} -> corr_p_val {val:.4f}")


DMN seed-DCPutamen 
DMN seed-DorsalCaudate 
DMN seed-DRPutamen 
DMN seed-InfVentralCaudate 
DMN seed-SupVentralCaudate 
DMN seed-VRPutamen 
salience seed-DCPutamen 
salience seed-DorsalCaudate 
salience seed-DRPutamen 
salience seed-InfVentralCaudate 
salience seed-SupVentralCaudate 
salience seed-VRPutamen 


# Extra code - maybe removed in the future